import numpy as np

SAMPLES = 3
STARTING_BALANCE = 10_000_000_000
WINNING_CHANCE = 0.5
START_BET = 1
GAMES = 1_000

def results_generator(n):
    for _ in range(n):
        yield

balances = np.full(SAMPLES, STARTING_BALANCE-START_BET, dtype=np.int64)
bets = np.full(SAMPLES, START_BET, dtype=np.int64)
results = np.random.choice([False, True], size=SAMPLES)

def game_summary():
    summary = f"balances: {balances}\nbets: {bets}\nresults: {results}\n"
    return summary

print(game_summary())

for game in range(GAMES):
    balances = np.where(results, balances + bets * 2, balances)
    bets = np.where(results, START_BET, bets * 2)
    balances -= bets
    results = np.random.choice([False, True], size=SAMPLES)
    print(game_summary())

In [54]:
import numpy as np

from functools import lru_cache
from abc import ABC, abstractmethod

class RouletteSimulation(ABC):
    def __init__(self, starting_balance, winning_chance, samples, start_bet):
        self.starting_balance = starting_balance
        self.winning_chance = winning_chance # Desc
        self.samples = samples
        self.start_bet = start_bet

        self.results = None
        self.bets = None
        self.balances = None

        self.current_game = 0

    def game_results(self):
        return np.random.choice(a=[True, False], size=self.samples, p=[self.winning_chance, 1 - self.winning_chance])


    def next_game(self):
        if self.current_game == 0:
                self.balances = np.full(self.samples, self.starting_balance, dtype=np.int64)
                self.bets = self.first_bets()
                self.balances -= self.bets
                self.results = self.game_results()
        self.current_game += 1

        self.balances = np.where(self.results, self.balance_after_win(), self.balances)
        self.bets = np.where(self.results, self.bets_after_win(), self.bets_after_lose())
        self.balances -= self.bets
        self.results = self.game_results()

    def __repr__(self):
        summary = f"balances: {self.balances}\nbets: {self.bets}\nresults: {self.results}\n"
        return summary

    def first_bets(self):
        return np.full(self.samples, self.start_bet, dtype=np.int64)

    def balance_after_win(self):
        return self.balances + self.bets * (1 / self.winning_chance)

    @abstractmethod
    def bets_after_win(self):
        pass
    @abstractmethod
    def bets_after_lose(self):
        pass



class Martingale(RouletteSimulation):
    def __init__(self, starting_balance, samples, start_bet):
        winning_chance = 0.5
        super().__init__(starting_balance, winning_chance, samples, start_bet)

    def bets_after_win(self):
        return self.first_bets()

    def bets_after_lose(self):
        return self.bets * 2

class Fibonacci(RouletteSimulation):
    def __init__(self, starting_balance, samples):
        winning_chance = 0.5
        start_bet = 1
        super().__init__(starting_balance, winning_chance, samples, start_bet)

        self.fib_n = np.full(samples, 0) # Based on fib.results

    @lru_cache(maxsize=None)
    def fib(self, n):
        if n < 2:
            return 1
        return self.fib(n - 2) + self.fib(n - 1)

    def fib_fill(self, fib_n: np.ndarray):
        print(np.vectorize(self.fib)(fib_n))
        return np.vectorize(self.fib)(fib_n)

    def next_game(self):
        super().next_game()
        self.fib_n = np.where(self.results, self.fib_n - 2, self.fib_n + 1)

    def bets_after_win(self):
        self.fib_n -= self.results * 2
        return self.fib_fill(self.fib_n - 2)

    def bets_after_lose(self):
        self.fib_n += self.results
        return self.fib_fill(self.fib_n + 1)

martingale = Martingale(starting_balance=10_000_000,
                        samples=3,
                        start_bet=1,
)
print(martingale)
for _ in range(1_0):
    martingale.next_game()
    print(martingale)

In [55]:
fibonacci = Fibonacci(starting_balance=10_000_000,samples=3)
print(fibonacci)
for _ in range(1_0):
    fibonacci.next_game()
    print(fibonacci)

balances: None
bets: None
results: None

[1 1 1]
[1 1 1]
balances: [10000000.  9999998.  9999998.]
bets: [1 1 1]
results: [False False False]

[1 1 1]
[1 1 1]
balances: [9999999. 9999997. 9999997.]
bets: [1 1 1]
results: [False False  True]

[1 1 1]
[1 1 1]
balances: [9999998. 9999996. 9999998.]
bets: [1 1 1]
results: [ True  True False]

[1 1 1]
[1 1 1]
balances: [9999999. 9999997. 9999997.]
bets: [1 1 1]
results: [ True  True False]

[1 1 1]
[1 1 1]
balances: [10000000.  9999998.  9999996.]
bets: [1 1 1]
results: [ True  True  True]

[1 1 1]
[1 1 1]
balances: [10000001.  9999999.  9999997.]
bets: [1 1 1]
results: [False False False]

[1 1 1]
[1 1 1]
balances: [10000000.  9999998.  9999996.]
bets: [1 1 1]
results: [False  True False]

[1 1 1]
[1 1 1]
balances: [9999999. 9999999. 9999995.]
bets: [1 1 1]
results: [False  True False]

[1 1 1]
[1 1 1]
balances: [ 9999998. 10000000.  9999994.]
bets: [1 1 1]
results: [ True  True  True]

[1 1 1]
[1 1 1]
balances: [ 9999999. 10000001.  99999